In [95]:
# Storage for annotationsfrom jupyter_bbox_widget import BBoxWidget
from jupyter_bbox_widget import BBoxWidget
import ipywidgets as widgets
import os
import json
import shutil
from IPython.display import display

In [96]:
# Paths
image_folder = r"RipNoRip/Rip"
processed_folder = os.path.join(image_folder, "Processed")
skip_folder = os.path.join(image_folder, "Skip")
annotations_path = "RipNoRip/rip_current_annotations.json"

# Ensure processed folder exists
os.makedirs(processed_folder, exist_ok=True)
os.makedirs(skip_folder, exist_ok=True)

# Load existing annotations if available
if os.path.exists(annotations_path):
    with open(annotations_path, "r") as f:
        annotations = json.load(f)
else:
    annotations = {}

In [97]:
# Function to get remaining images
def get_image_files():
    return sorted([f for f in os.listdir(image_folder) 
                   if f.endswith(('.jpg', '.png', '.tif'))])

# Initialise image list
image_files = get_image_files()

# Check if images exist
if not image_files:
    print("No new images to annotate in", image_folder)
else:
    print(f"Found {len(image_files)} unprocessed images.")

Found 12 unprocessed images.


In [98]:
# Progress bar - doesn't seem to work
w_progress = widgets.IntProgress(value=0, max=len(image_files), description="Progress")

# Initialize annotation widget
if image_files:
    w_bbox = BBoxWidget(
        image=os.path.join(image_folder, image_files[0]),  
        classes=["Rip Current"]
    )
else:
    w_bbox = BBoxWidget(classes=["Rip Current"])  # Empty widget if no images found

# Combine widgets
w_message = widgets.HTML("")
w_container = widgets.VBox([w_progress, w_bbox, w_message])

In [99]:


# Function to move an image and update list
def move_image(image_file, destination_folder):
    src_path = os.path.join(image_folder, image_file)
    dst_path = os.path.join(destination_folder, image_file)
    shutil.move(src_path, dst_path)

    # Refresh remaining images list
    global image_files
    image_files = get_image_files()
    w_progress.max = len(image_files)

# Function to finish annotation process
def finish_annotation():
    w_message.value = "<b>🎉 Congratulations! All images have been processed. 🎉</b>"
    w_container.children = (w_message,)  # Remove all widgets except the message
    print("Annotations saved. Closing widget.")
    
# Function to load the next image
def load_next_image():
    if image_files:
        w_bbox.image = os.path.join(image_folder, image_files[0])
        w_bbox.bboxes = []  # Reset bounding boxes
    else:
        finish_annotation()
        
# Function to skip image
@w_bbox.on_skip
def skip():
    if not image_files:
        return

    image_file = image_files[0]  # Get first image in list
    move_image(image_file, skip_folder)  # Move skipped image

    # Update progress bar AFTER the image is skipped
    w_progress.value += 1
    load_next_image()

# Function to submit annotation
@w_bbox.on_submit
def submit():
    if not w_bbox.bboxes:  # Check if the annotation list is empty
        w_message.value = "<b style='color: red;'>⚠️ Please annotate before submitting!</b>"
        return  # Stop function execution

    image_file = image_files[0]
    annotations[image_file] = w_bbox.bboxes

    # Save annotations to file
    with open(annotations_path, "w") as f:
        json.dump(annotations, f, indent=4)

    # Move image to processed folder
    move_image(image_file, processed_folder)

    # Update progress bar AFTER the image is processed
    w_progress.value += 1

    # Clear error message if any
    w_message.value = ""

    # Load next image
    load_next_image()

In [100]:
# Display annotation UI
display(w_container)

In [101]:
with open(annotations_path, 'r') as f:
    print(f.read())

{
    "RGB_NZ002_20151221_2226.png": [
        {
            "x": 33,
            "y": 42,
            "width": 41,
            "height": 40,
            "label": "Rip Current"
        }
    ],
    "RGB_NZ002_20160103_2236.png": [
        {
            "x": 25,
            "y": 41,
            "width": 50,
            "height": 39,
            "label": "Rip Current"
        },
        {
            "x": 126,
            "y": 161,
            "width": 58,
            "height": 33,
            "label": "Rip Current"
        }
    ],
    "RGB_NZ002_20160123_2238.png": [
        {
            "x": 13,
            "y": 35,
            "width": 74,
            "height": 39,
            "label": "Rip Current"
        },
        {
            "x": 127,
            "y": 151,
            "width": 58,
            "height": 40,
            "label": "Rip Current"
        }
    ],
    "RGB_NZ002_20160219_2225.png": [
        {
            "x": 126,
            "y": 155,
            "width": 62,
    